In [ ]:
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#data = np.load('Samples/MG17.npy')
#data = np.genfromtxt('Samples/MG17.csv')
#data = data.reshape((data.shape[0],1))
#from scipy import stats
#data = stats.zscore(data)

testlen = 143*2#150+250#143 #lyapunov

data = np.load('Samples/MG17.npy')
data = data.reshape((data.shape[0],1))
data = data[:2801,:]
#data = data[:5801,:]
from scipy import stats
data = stats.zscore(data)
test = data[2300+1:2300+testlen+1,:]
#test = data[5300+1:5300+1+testlen,:]
seeds = np.genfromtxt('seeds.csv')
#print(np.where(seeds==931833149))
#brekn()


a = np.load('Samples/MG17.npy')
a = data.reshape((data.shape[0],1))
a = data[:2801,:]
normMean = np.mean(a[:2801])
normStd = np.std(a[:2801])
#a = data[:5801,:]
#normMean = np.mean(a[:5801])
#normStd = np.std(a[:5801])


a = a[2301:2301+testlen,:]
#a = a[5301:5301+testlen,:]

gamma = np.asarray([0, .125, .25, .28125, .3125, .34375, .375, .5, .75, 1]) # Leakage

gamma = np.asarray([0, .125, .1875, .25, .3175-.03125, .3175, .3175+.03125, .375, .5, .75, 1])

beta = np.asarray([1e-6,1.05e-6,1.1e-6,1.1e-6*.95, 1.1e-6*1.05])
beta1 = np.logspace(-10,.3,10)
beta2 = np.logspace(-8,-6,10)
beta3 = np.linspace(1e-6*.9,1e-6*1.1,5)

beta4 = np.asarray([1e-6,1.05e-6,1.1e-6,1.1e-6*.95, 1.1e-6*1.05])
beta5 = np.linspace(1.1e-6*.9,1.1e-6*1.1,5)

beta = np.zeros((35,))
beta[:10]   = beta1
beta[10:20] = beta2
beta[20:25] = beta3
beta[25:30]   = beta4 
beta[30:]   = beta5  

rho = np.sort(beta2)

r2s = np.zeros((len(rho), 100))#((15,100))
r2sci = np.zeros((len(rho), 100))

nrmses = np.zeros((len(rho), 100))#((15, 100))
nrmseStd = np.zeros((len(rho),1))
stds = np.zeros((len(rho),1))
stdsr2 = np.zeros((len(rho),1))

mses = np.zeros((len(rho), 100))#((15, 100))
#for i in range(rho.shape[0]):
#    print(rho[i])
outColl = np.zeros((rho.shape[0],100, testlen))
outColl2 = np.zeros((rho.shape[0],100, testlen))
outCollNRMSE = np.zeros((rho.shape[0],100, testlen))


overall = np.zeros((rho.shape[0], 1))
overallr2 = np.zeros((rho.shape[0], 1))
overallr2sci = np.zeros((rho.shape[0], 1))
overallNRMSE = np.zeros((rho.shape[0], 1))

completeMSE = np.zeros((rho.shape[0], 100))
completeR2 = np.zeros((rho.shape[0], 100))
completeR2sci = np.zeros((rho.shape[0], 100))
completeNRMSE = np.zeros((rho.shape[0], 100))

neurons = '2048'
param = 'beta'

mean = np.mean(test)
meanCurr = np.mean(test, axis=0)

for i in range(rho.shape[0]):
    for j in range(100):#100):#seeds.shape[0]):
        w = np.load('MGBeta/%sWout2%s_with%s_at%s.npy' %(neurons, param, rho[i], seeds[j]))

        out = np.dot(w, np.hstack((np.ones((500,1)),np.load('MGBeta/pred/%sintPred2%s_with%s_at%s.npy' % (neurons, param, rho[i], seeds[j]) ))).T  ).T

        out = out[:testlen,:]
        outColl[i,j] = out[:,0].reshape((testlen))

        
        sumt = 0
        sumo = 0
        for m in range(test.shape[1]):
            
            t = np.sum((test[:,m]-meanCurr[m])**2)
            sumt = sumt+t
            o = np.sum((out[:,m]-meanCurr[m])**2)
            sumo = sumo+o
        sumo = sumo/test.shape[1]
        sumt = sumt/test.shape[1]
        r2 = sumo/sumt
        #print(test.shape)
        #print(out.shape)
        r2 = 1-(np.sum((test-out)**2)/np.sum((test-mean)**2))
        r2sci[i,j] = r2_score(test, out)
        mse = np.mean((test-out)**2)
        r2s[i,j] = r2
        mses[i,j] = mse
        out2 = out.copy()
        test2 = test.copy()
        for n in range(1):
            out2[:,n] = (out[:,n]*normStd)+normMean
            test2[:,n] = (test[:,n]*normStd)+normMean
        nrmses[i,j] = np.sqrt(np.mean((a-out2)**2))/np.abs(np.mean(a))
        #print(nrmses[i,j])
        r2s[i,j] = r2_score(a,out2)
        outColl[i,j] = out2[:,0].reshape((testlen))
    stds[i] = np.std(mses[i,:])
    #stdsr2[i] = np.std(r2s[i,:])
    stdsr2[i] = np.std(r2sci[i,:])
    nrmseStd[i] = np.std(nrmses[i,:])
    
    fig = plt.figure()
    plt.plot(np.mean(outColl[i], axis=0), label = 'pred')
    plt.plot(a[:], label = 'target')
    plt.legend()
    plt.xlabel('Zeitschritte')
    plt.ylabel('Wert')
    plt.title('X')
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    y = np.mean(outColl[i], axis=0)[17:]
    ax.plot(y, np.mean(outColl[i], axis=0)[:-17], zs=0, zdir='y')

    y = a[17:]
    ax.plot(y, a[:-17], zs=0, zdir='y')

    ax.legend(['Free-running predicted signal', 'Target signal' ])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

    ind = np.argpartition(mses[i], 10)[:10]
    overall[i] = np.median(mses[i])
    overallr2[i] = np.median(r2s[i])
    overallr2sci[i] = np.median(r2sci[i])
    overallNRMSE[i] = np.median(nrmses[i])

np.save('Plot2/complete/MGlt/Complete2%sMSE_MG_%s_%s_shifted_long1.npy' % (testlen, neurons, param), mses)
np.save('Plot2/complete/MGlt/Complete2%sR2_MG_%s_%s_shifted_long1.npy' % (testlen, neurons, param), r2s)
np.save('Plot2/complete/MGlt/Complete2%sR2sci_MG_%s_%s_shifted_long1.npy' % (testlen, neurons, param), r2sci)
np.save('Plot2/complete/MGlt/Complete2%sNRMSE_MG_%s_%s_shifted_long1.npy' % (testlen, neurons, param), nrmses)
print(overallr2)

print('over ', testlen-150, ' steps')
print('MSE: ', overall)
print(np.argmin(overall), ' ', rho[np.argmin(overall)])


print('SK: ', overallr2sci)
print('SK Denormed: ', overallr2)
print('NRMSE: ', overallNRMSE)


print(np.argmin(overall), ' ', rho[np.argmin(overall)])
print(stds.shape)
print(nrmses.shape)
data = np.zeros((100,rho.shape[0]))
for i in range(1, rho.shape[0]):
    data[:,i] = nrmses[i]

dataset = np.median(nrmses, axis=1)
#print(dataset.shape)
spread = np.zeros((rho.shape[0],1))
for i in range(rho.shape[0]):
    spread[i]= np.std(nrmses[i,:])

spreadR2 = np.zeros((rho.shape[0],1))
datasetR2 = np.median(r2s, axis=1)
for i in range(rho.shape[0]):
    spreadR2[i]= np.std(r2sci[i,:])



print('shape: ', nrmses.shape)
dataset = np.median(nrmses, axis=1)
print(dataset.shape)
spread = np.zeros((rho.shape[0],1))
for i in range(rho.shape[0]):
    spread[i]= 0#np.std(nrmses[i,:])

spreadR2 = np.zeros((rho.shape[0],1))
datasetR2 = np.median(r2sci, axis=1)
for i in range(rho.shape[0]):
    spreadR2[i]= 0#np.std(r2sci[i,:])

fig, ax1 = plt.subplots(figsize=(20, 10))
#plt.xlim((.6,1.5))
#ax1.set_ylim((0, 10))
color = 'tab:red'
if param == 'sparW':
    ax1.set_xlabel(r'$spar(W^{res})$')
elif param == 'train':
    ax1.set_xlabel(r'$S_T$')
else: 
    ax1.set_xlabel(r'$\%s$' % (param))
ax1.set_ylabel('NRMSE', color=color)
ax1.errorbar(rho, dataset,capsize=15,label = 'NRMSE', color =color, linewidth=3)


ax2 = ax1.twinx()  
from matplotlib.ticker import FormatStrFormatter
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))


color = 'tab:blue'
ax2.set_ylabel('$R^2$', color=color)  
size = datasetR2.shape
ax2.errorbar(rho, datasetR2,capsize=15, label = '$R^2$', linewidth=3, color=color)
offset = 1

plt.rcParams.update({'font.size': 54})

plt.rcParams.update({'errorbar.capsize': 12})
fig.tight_layout()  

plt.savefig('pdfs/%s%s_MG%s_shifted1.pdf' % (neurons, param, testlen), bbox_inches = 'tight',pad_inches = 0.1)

plt.show()
